# Prediction

In [1]:
import requests, json
import numpy as np
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

## Update below variables as per your environment.

In [ ]:
import requests
EZAF_ENV = "hpe-staging-ezaf" 
token_url = f"https://keycloak.{EZAF_ENV}.com/realms/UA/protocol/openid-connect/token"

data = {
    "username" : input("Enter username."),
    "password" : input("Enter password."),
    "grant_type" : "password",
    "client_id" : "ua-grant",
}

token_responce = requests.post(token_url, data=data, allow_redirects=True, verify=False)

token = token_responce.json()["access_token"]

In [2]:
KUBEFLOW_URL = "kubeflow.hpe-ezaf.com"
DEPLOYMENT_NAME = "loan-approval-1"
NAMESPACE="hpedemo-user01"
SVC = f'{DEPLOYMENT_NAME}-predictor-default.{NAMESPACE}.svc.cluster.local'

URL = "http://{}/v2/models/{}/infer".format(SVC, DEPLOYMENT_NAME)

In [3]:
PREDICTION_URL = KUBEFLOW_URL#"{}/seldon/{}/{}/api/v1.0/predictions".format(KUBEFLOW_URL,NAMESPACE,SELDON_DEPLOYMENT_NAME)
print(URL)

http://loan-approval-1-predictor-default.hpedemo-user01.svc.cluster.local/v2/models/loan-approval-1/infer


In [5]:

headers = {
    'Authorization': f"Bearer {token}",
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

Approved = {'Dependents': 0.0,
 'LoanAmount': 105000.0,
 'Loan_Amount_Term': 360.0,
 'Credit_History': 1.0,
 'Property_Area': 2.0,
 'Credit_History_Income_Sum': 5550331.91998864,
 'Dependents_LoanAmount_Sum': 72780000.0,
 'Dependents_EMI_mean': 1121.6651819119277,
 'Loan_Amount_Term_per_Total_Income': 0.06154898273209095,
 'EMI_per_Loan_Amount_Term': 2.4498217625000005,
 'EMI_per_LoanAmount': 0.008399388900000002,
 'Property_Area_LoanAmount_per_Total_Income_mean': 23.36319041854885,
 'Loan_Amount_Term_Bins': 1.0,
 'Total_Income_Bins': 2.0,
 'LoanAmount_per_Total_Income_Bins': 1.0,
 'Loan_Amount_Term_per_Total_Income': 0.125,
 'Gender_Male': 1.0,
 'Married_Yes': 0.0,
 'Education_Not Graduate': 0.0,
 'Self_Employed_Yes': 0.0
}

Declined = {'Dependents': 0.0,
 'LoanAmount': 74000.0,
 'Loan_Amount_Term': 360.0,
 'Credit_History': 1.0,
 'Property_Area': 2.0,
 'Credit_History_Income_Sum': 5550331.91998864,
 'Dependents_LoanAmount_Sum': 72780000.0,
 'Dependents_EMI_mean': 1121.6651819119277,
 'Loan_Amount_Term_per_Total_Income': 0.09130103981739791,
 'EMI_per_Loan_Amount_Term': 1.7265410516666668,
 'EMI_per_LoanAmount': 0.0083993889,
 'Property_Area_LoanAmount_per_Total_Income_mean': 23.36319041854885,
 'Loan_Amount_Term_Bins': 1.0,
 'Total_Income_Bins': 1.0,
 'LoanAmount_per_Total_Income_Bins': 1.0,
 'Gender_Male': 1.0,
 'Married_Yes': 0.0,
 'Education_Not Graduate': 0.0,
 'Self_Employed_Yes': 0.0
}

In [6]:
inputs = {
  "inputs": [
    { 
      "name": "ndarray",
      "shape": [2,19],
      "datatype": "FP32",
      "data": [
        list(Approved.values()),
        list(Declined.values())  
      ]
    }
  ],
    "outputs": [
        {
            "name": "predict"
        },
        {
            "name": "predict_proba"
        }
    ]
}

In [7]:
json_request = json.dumps(inputs)
# print(json_request)
response = requests.post(URL, headers=headers, data=json_request, verify=False)
# print(response)
# print(response.json())
outputs = response.json()["outputs"]
print(outputs)

[{'name': 'predict', 'shape': [2], 'datatype': 'FP64', 'parameters': None, 'data': [1.0, 0.0]}, {'name': 'predict_proba', 'shape': [2, 2], 'datatype': 'FP64', 'parameters': None, 'data': [0.3284196527929984, 0.6715803472070015, 0.6996414316032933, 0.3003585683967066]}]


# Approved

In [8]:
status = outputs[0]["data"][0]
probability = np.array(outputs[1]["data"])[1]

output = {"probability": probability, "Loan_Status": status }
print("Prediction: {}\nProbability: {:.2f}".format(output["Loan_Status"],output["probability"]))

Prediction: 1.0
Probability: 0.67


# Declined

In [9]:
status = outputs[0]["data"][1]
probability = np.array(outputs[1]["data"])[3]

output = {"probability": probability, "Loan_Status": status }
print("Prediction: {}\nProbability: {:.2f}".format(output["Loan_Status"],output["probability"]))

Prediction: 0.0
Probability: 0.30
